# Combining and Merging

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

Data contained in pandas objects can be combined in a number of ways:
1. `pandas.merge`: connects rows in DataFrames based on one or more keys.
1. `pandas.concat`: concatenate or _stack_ objects along an axis.
1. `combine_first`: splice together overlapping data to fill in missing values in one object with values from another

## Database-Style DataFrame Joins
- `pandas.merge`: main entry point to achieve joins in DataFrames

In [2]:
df1 = pd.DataFrame({'key': list('bbacaab'),
                    'data1':pd.Series(range(7), dtype="Int64")})
df2 = pd.DataFrame({"key": ['a','b','d'],
                    'data2':pd.Series(range(3), dtype="Int64")})                    

In [3]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [4]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


- An exmaple of many-to-one join.
- data in df1 has multiple rows labeled a and b,
- whereas, df2 has only one row for each value in the key column.

In [5]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


- if we do not specify which column to join on, then `pandas.merge` uses the overlapping column anmes as the keys.
- Its a good practice to always specify the keys.

In [6]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [12]:
df3 = pd.DataFrame({'lkey': list('bbacaab'),
                    'data1':pd.Series(range(7), dtype='Int64')})
df4 = pd.DataFrame({'rkey':list('abd'),
                    'data2':pd.Series(range(3), dtype='Int64')})

In [13]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,a,2,a,0
3,a,4,a,0
4,a,5,a,0
5,b,6,b,1


- By default `pandas.merge` does inner join.
- other possible options are
    - `left`
    - `right`
    - `outer`: takes the uniun of keys, combining the effect of applying both left and right jons.

In [9]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,a,2,0
1,a,4,0
2,a,5,0
3,b,0,1
4,b,1,1
5,b,6,1
6,c,3,<NA>
7,d,<NA>,2


In [14]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey', how='outer')

,lkey,data1,rkey,data2
0,a,2,a,0
1,a,4,a,0
2,a,5,a,0
3,b,0,b,1
4,b,1,b,1
5,b,6,b,1
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


- _Many-to-many_ merges form the Cartesian product of the matching keys.

In [19]:
df1 = pd.DataFrame({'key':list('bbacab'),'data1':pd.Series(range(6), dtype='Int64')})
df2 = pd.DataFrame({'key':list('ababd'),'data2':pd.Series(range(5), dtype='Int64')})

In [16]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [20]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [21]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


In [22]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,a,4,0
7,a,4,2
8,b,5,1
9,b,5,3


Merging with mutliple keys

In [25]:
left = pd.DataFrame({'key1':['foo','foo','bar',],'key2':['one','two','one',],"lval": pd.Series([1,2,3], dtype='Int64')})
right = pd.DataFrame({'key1':['foo','foo','bar','bar',],'key2':['one','one','one','two',],"rval": pd.Series([4,5,6,7], dtype='Int64')})

In [26]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [27]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [33]:
pd.merge(left, right, on=['key1','key2'], how='outer')

,key1,key2,lval,rval
0,bar,one,3,6
1,bar,two,<NA>,7
2,foo,one,1,4
3,foo,one,1,5
4,foo,two,2,<NA>


In [34]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [35]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


## Merging on Index
- Some cases, the merge key(s) in a DF will be on its index(row labels).
- we must pass `left_index=True` or `right_index=True` (or both) to indicate index(rows) must be used as the merge key

In [36]:
left1 = pd.DataFrame({'key':list('abaabc'), 'value':pd.Series(range(6),dtype='Int64')})
right1 = pd.DataFrame({'group_val':[3.5,7]}, index=['a','b'])

In [37]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [38]:
right1

,group_val
a,3.5
b,7.0


In [39]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0


Indexes on left are preserved

In [40]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


- With hierarchical indexed data, things are more complicatd, as joining on index is equivalent to a multiple-key merge

In [41]:
lefth = pd.DataFrame({"key1":['Ohio','Ohio','Ohio','Nevada','Nevada',],
                        'key2':[2000,2001,2002,2001,2002],'data':pd.Series(range(5), dtype='Int64')})

In [42]:
righth_index = pd.MultiIndex.from_arrays(
    [
        ['Nevada','Nevada','Ohio','Ohio','Ohio','Ohio',],
        [2001,2000,2000,2000,2001,2002]
    ]
)

In [43]:
righth = pd.DataFrame({'event1':pd.Series([0,2,4,6,8,10], dtype='Int64',index=righth_index), 
                    'event2':pd.Series([1,3,5,7,9,11], dtype='Int64', index=righth_index)}, )

In [44]:
lefth

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [45]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

- In this case we have to indicate multiple columns to merge on as a list.
- note the handling of duplicate index values with `how='outer'`

In [46]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


In [47]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

,key1,key2,data,event1,event2
4,Nevada,2000,<NA>,2,3
3,Nevada,2001,3,0,1
4,Nevada,2002,4,<NA>,<NA>
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11


In [48]:
left2 = pd.DataFrame([[1.,2.],[3.,4.],[5.,6.]],
                    index=list('ace'),
                    columns=['Ohio', 'Nevada']).astype('Int64')

In [49]:
right2 = pd.DataFrame([[7.,8.],[9.,10.],[11.,12.],[13,14]],
                    index=list('bcde'),
                    columns=['Missouri','Alabama']).astype('Int64')

In [50]:
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [51]:
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [52]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [53]:
left2.join(right2)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
c,3,4,9,10
e,5,6,13,14


In [54]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [56]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [57]:
right1

,group_val
a,3.5
b,7.0


In [58]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


- For simple index-on-index merges, you can pass a list of DataFrames to join as an alternative to using the more general `pandas.concat` function

In [59]:
another= pd.DataFrame([[7.,8.],[9.,10.,],[11.,12.],[16.,17.]],
                    index=list('acef'),
                    columns=['New York', 'Oregon'])

In [60]:
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [61]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0


In [62]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0
b,<NA>,<NA>,7,8,NaN,NaN
d,<NA>,<NA>,11,12,NaN,NaN
f,<NA>,<NA>,<NA>,<NA>,16.0,17.0
